In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.datasets import load_boston
from sklearn.metrics import r2_score, mean_squared_log_error, mean_squared_error
import datetime as dt
import pickle
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split

In [2]:
data=pd.read_pickle("full_dataframe_20210803_150443.pkl")
data.reset_index(inplace=True)
data.drop(['index','event_id'], inplace=True, axis=1)
print(data.shape)
data.head()

(7680, 531)


,MISS_DISTANCE,RELATIVE_SPEED,RELATIVE_POSITION_R,RELATIVE_POSITION_T,RELATIVE_POSITION_N,RELATIVE_VELOCITY_R,RELATIVE_VELOCITY_T,RELATIVE_VELOCITY_N,COLLISSION_PROBABILITY,OBJECT1_CR_R,...,OBJECT2_CORR_CTDOT_R_4,OBJECT2_CORR_CTDOT_T_4,OBJECT2_CORR_CTDOT_N_4,OBJECT2_CORR_CTDOT_RDOT_4,OBJECT2_CORR_CNDOT_R_4,OBJECT2_CORR_CNDOT_T_4,OBJECT2_CORR_CNDOT_N_4,OBJECT2_CORR_CNDOT_RDOT_4,OBJECT2_CORR_CNDOT_TDOT_4,COLLISSION_PROBABILITY_TARGET
0,18753.0,14347.0,-700.1,-5172.4,18012.1,14.4,-13791.4,-3957.2,-30.000000,22.474480,...,-0.999926,0.378319,-0.845081,-0.301615,0.901438,-0.372185,0.971804,0.258500,-0.901937,-30.000000
1,23982.0,13574.0,22.3,10104.3,21749.5,-39.7,-12310.9,5718.9,-30.000000,78.188040,...,-0.999511,-0.675979,-0.776708,0.667410,0.911423,0.652982,0.860347,-0.641016,-0.911900,-30.000000
2,23709.0,12093.0,188.2,-13921.3,19191.2,19.7,-9788.5,-7101.8,-30.000000,31.349310,...,-0.999995,-0.973987,0.792263,0.961697,-0.517752,-0.543370,0.926369,0.531127,0.517949,-27.650917
3,314.0,2001.0,-18.4,-311.4,-42.7,0.5,-268.6,1983.8,-16.843451,24.527640,...,-0.999942,-0.121330,-0.007675,0.161742,0.087987,0.034350,0.360326,-0.070074,-0.088322,-30.000000
4,10978.0,10027.0,132.6,-8226.7,-7268.6,8.8,-6638.6,7515.2,-30.000000,1.362408,...,-0.999340,0.352616,-0.301764,-0.332939,0.229495,-0.077586,0.700588,0.085000,-0.229904,-30.000000


In [3]:
data=data.head(200)

In [4]:
data.shape

(200, 531)

In [5]:
train, test = train_test_split(data, test_size=0.20, random_state=42)

In [6]:
Y_train = train["COLLISSION_PROBABILITY_TARGET"]
X_train= train.drop(["COLLISSION_PROBABILITY_TARGET"], axis=1)
Y_test = test["COLLISSION_PROBABILITY_TARGET"]
X_test= test.drop(["COLLISSION_PROBABILITY_TARGET"], axis=1)


In [7]:
X = X_train
y = Y_train

In [8]:
def bayesian_opt_lgbm(X, y, init_iter=3, n_iters=7, random_state=11, seed = 101, num_iterations = 100):
      dtrain = lgb.Dataset(data=X, label=y)
      def lgb_r2_score(preds, dtrain):
            labels = dtrain.get_label()
            return 'r2', r2_score(labels, preds), True
      # Objective Function
      def hyp_lgbm(num_leaves, feature_fraction, learning_rate, bagging_fraction, max_depth, min_split_gain, min_child_weight):
              params = {      'application':'regression',
                              'num_iterations': num_iterations,
                              'early_stopping_round':50,
                              'verbose':-1,
                              'metric':'lgb_r2_score'} # Default parameters
              params["num_leaves"] = int(round(num_leaves))
              params["learning_rate"] = learning_rate
              params['feature_fraction'] = max(min(feature_fraction, 1), 0)
              params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
              params['max_depth'] = int(round(max_depth))
              params['min_split_gain'] = min_split_gain
              params['min_child_weight'] = min_child_weight
              cv_results = lgb.cv(params, dtrain, nfold=5, seed=seed,categorical_feature=[], stratified=False,
                                  verbose_eval =None, feval=lgb_r2_score)
              #print(cv_results)
              return np.max(cv_results['r2-mean'])
    
              # Domain space-- Range of hyperparameters 
      pds = {     'num_leaves': (80, 120),
                  'feature_fraction': (0.1, 0.9),
                  'bagging_fraction': (0.8, 1),
                  'max_depth': (8, 15),
                  'learning_rate':(0.001,0.05), 
                  'min_split_gain': (0.001, 0.1),
                  'min_child_weight': (10, 25)
                  }
      # Surrogate model
      optimizer = BayesianOptimization(hyp_lgbm, pds, random_state=random_state)
                                          
      # Optimize
      optimizer.maximize(init_points=init_iter, n_iter=n_iters)

      return optimizer

bayesian_ouput=bayesian_opt_lgbm(X, y, init_iter=5, n_iters=10, random_state=77, seed = 101, num_iterations = 200)

|   iter    |  target   | baggin... | featur... | learni... | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.8145   |  0.9838   |  0.6138   |  0.03793  |  8.975    |  11.31    |  0.07901  |  93.05    |
|  2        |  0.7647   |  0.9082   |  0.2922   |  0.02773  |  10.8     |  20.73    |  0.08383  |  103.5    |
|  3        |  0.7775   |  0.8592   |  0.3248   |  0.03557  |  10.96    |  10.86    |  0.07496  |  98.09    |
|  4        |  0.627    |  0.8352   |  0.1395   |  0.01533  |  8.468    |  21.27    |  0.007313 |  97.28    |
|  5        |  0.7362   |  0.8728   |  0.2216   |  0.02779  |  11.1     |  10.54    |  0.08247  |  90.93    |
|  6        |  0.8154   |  1.0      |  0.9      |  0.05     |  8.0      |  10.0     |  0.1      |  120.0    |
|  7        |  0.7363   |  1.0      |  0.9      |  0.05     |  15.0     |  25.0     |  0.001    |  120.0    |
|  8      

In [9]:
opt_parameters=bayesian_ouput.max['params']
opt_parameters

{'bagging_fraction': 1.0,
 'feature_fraction': 0.9,
 'learning_rate': 0.05,
 'max_depth': 15.0,
 'min_child_weight': 10.0,
 'min_split_gain': 0.001,
 'num_leaves': 115.77094520211915}

In [ ]:
filename="opt_parameters_{}.pkl".format(dt.datetime.now().strftime("%Y%m%d_%H%M%S"))
a_file = open(filename, "wb")

pickle.dump(opt_parameters, a_file)

a_file.close()

In [ ]:
a_file = open(filename,"rb")
output = pickle.load(a_file)
output

In [ ]:
#optimizer.max['params']

In [ ]:
#'bagging_fraction': 1.0, 'feature_fraction': 0.9, 'max_depth': 8.0, 'min_child_weight': 25.0, 'min_split_gain': 0.013771321931506838, 'num_leaves': 88.93816438820497}

In [10]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'l2',
    'learning_rate': opt_parameters.get("learning_rate"),
    'feature_fraction': opt_parameters.get("feature_fraction"),
    'bagging_fraction': opt_parameters.get("bagging_fraction"),
    #'bagging_freq': 10,
    'verbose': -1,
    "max_depth": int(round(opt_parameters.get("max_depth"))),
    "num_leaves": int(round(opt_parameters.get("num_leaves"))),  
    #"max_bin": 512,
    'min_split_gain' : opt_parameters.get("min_split_gain"),
    "num_iterations": 200,
    "n_estimators": 10,
    'min_child_weight' : opt_parameters.get("min_child_weight")
}

In [11]:
#Construct a gradient boosting model.
gbm = lgb.LGBMRegressor(**hyper_params)

In [12]:
lgbm_train = lgb.Dataset(X, label=y)
lgbm_eval = lgb.Dataset(X_test, label=Y_test,reference=lgbm_train)

In [13]:
gbm = lgb.train(params=hyper_params,
                train_set=lgbm_train,
                valid_sets=lgbm_eval,
                verbose_eval=20,
                #eval_metric='lgb_r2_score',
                early_stopping_rounds=100)

Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[10]	valid_0's l2: 32.6997


In [14]:
#Build a gradient boosting model from the training set (X, y)
""" gbm.fit(X, y,
        eval_set=[(X_test, Y_test)],
        eval_metric='l1',
        early_stopping_rounds=50) """


" gbm.fit(X, y,\n        eval_set=[(X_test, Y_test)],\n        eval_metric='l1',\n        early_stopping_rounds=50) "

In [15]:
Y_pred = gbm.predict(X_train, num_iteration=gbm.best_iteration)

In [16]:
print('The r2 of prediction is:', r2_score(y, Y_pred))
print('The MSE of prediction is:', mean_squared_error(y, Y_pred, squared=True))
print('The RMSE of prediction is:', mean_squared_error(y, Y_pred, squared=False))

The r2 of prediction is: 0.571129009601562
The MSE of prediction is: 29.122449334979812
The RMSE of prediction is: 5.3965219665058175


In [17]:
aux_y=pd.DataFrame(y)
aux_y.reset_index(inplace=True)
aux_y.drop(['index'], inplace=True, axis=1)
aux_y_pred=pd.DataFrame(Y_pred)
aux_y_pred.reset_index(inplace=True)
aux_y_pred.drop(['index'], inplace=True, axis=1)
frames=[aux_y,aux_y_pred]
result=pd.concat(frames,axis=1)
result

,COLLISSION_PROBABILITY_TARGET,0
0,-14.245728,-25.232460
1,-10.863279,-18.606522
2,-23.480697,-25.583255
3,-11.818728,-18.606522
4,-30.000000,-27.393545
...,...,...
155,-30.000000,-27.440501
156,-30.000000,-27.473499
157,-30.000000,-27.487137
158,-7.299383,-18.606522
